This notebook performs a comprehensive becnhmark suit for the inference procedure for the double pendulum system using the RxInfer framework.

_Author: Dmitry Bagaev_

In [1]:
using DrWatson

In [2]:
@quickactivate "RxInferThesisExperiments"

In [28]:
using RxInferThesisExperiments, RxInfer, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames

In [4]:
# Pregenerate paths for benchmark data
mkpath(datadir("nlds", "rxinfer"));

In [5]:
# Create default environment with default parameters
const environment = DoublePendulum()

DoublePendulum()

In [6]:
# Define state-transition function, uses RK4 method internally, see the `src/` folder
f(state) = state_transition(environment)(state)

f (generic function with 1 method)

In [46]:
# Include the model specification
include(srcdir("models", "rxinfer", "doublependulum.jl"));

In [58]:
function run_benchmark(params)
    @unpack T, niterations, seed = params
    
    states, observations = rand(StableRNG(seed), environment, T);
    model    = double_pendulum(T)
    result   = run_inference(model, observations; iterations = niterations)
    e_states = result.posteriors[:s]
    amse     = compute_amse(states, e_states)
    
    benchmark_modelcreation = @benchmark RxInfer.create_model(double_pendulum($T), 
        meta = double_pendulum_meta(), 
        constraints = double_pendulum_constraints(),
    )
    
    benchmark_inference = @benchmark run_inference(model, observations; iterations = $niterations) setup=begin
        model = double_pendulum($T)
        states, observations = rand(StableRNG($seed), environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local states, observations = rand(StableRNG(_seed), environment, T; random_start = true);
        local model    = double_pendulum(T)
        local result   = run_inference(model, observations; iterations = niterations)
        local e_states = result.posteriors[:s]
        return compute_amse(states, e_states)
    end
    
    output = @strdict T niterations seed states e_states observations amse emse benchmark_modelcreation benchmark_inference
    
    return output
end

run_benchmark (generic function with 1 method)

In [59]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"           => [ 100 ],
    "niterations" => [ 3, 5 ],
    "seed"        => [ 42 ]
));

In [60]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    path = datadir("nlds", "rxinfer")
    result, _ = produce_or_load(run_benchmark, path, params; tag = false, force = true)
    return result
end;

[ Info: Producing file /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer/T=100_niterations=3_seed=42.jld2 now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer/T=100_niterations=3_seed=42.jld2 saved.
[ Info: Producing file /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer/T=100_niterations=5_seed=42.jld2 now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer/T=100_niterations=5_seed=42.jld2 saved.


In [65]:
prepare_benchmarks_table(datadir("nlds", "rxinfer"))

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/thesis/data/nlds/rxinfer for result files.
[ Info: Added 2 entries.


Row,T,seed,niterations,amse,emse,inference,creation
,Int64?,Int64?,Int64?,Float64?,Float64?,Tuple…?,Tuple…?
1,100,42,3,4.12064,1.23184,"(""12.4942ms"", ""18.7074ms"", ""12.4942ms"")","(""4.8569ms"", ""6.1765ms"", ""4.8569ms"")"
2,100,42,5,4.10658,1.2254,"(""14.1548ms"", ""20.9859ms"", ""14.1548ms"")","(""4.8826ms"", ""6.2287ms"", ""4.8826ms"")"
